# Simulation - New

In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import uuid
from pandarallel import pandarallel
from tqdm.notebook import tqdm
from copy import deepcopy

pandarallel.initialize()

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
from mc_moea import *

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read position data

In [3]:
data = []
fnames = glob.glob("position/*.dat")
for fname in fnames:
    X = pickle.load(open(fname, "rb"))
    data.append(X)

### Define pattern's probability

In [4]:
p = [.32, .25, .17, .14, .09, .03]

### Stations

In [5]:
station_types

,types,frequency,capacity,cost
0,900 MHz Type I,900,800,1150000
1,900 MHz Type II,900,1200,1500000
2,1800 MHz Type I,1800,850,880000
3,1800 MHz Type II,1800,1250,1220000
4,2600 MHz Type I,2600,800,950000
5,2600 MHz Type II,2600,1300,1350000


### Simulation parameters

In [6]:
max_station = 20
n_population = 300
crossover_p = 0.3
mutation_p = 0.1
elitism_ratio = 0.5
trim_policy = "best"  # "random" or "best"
num_iters = 400
checkpoint_every = 20

### Initial population

In [7]:
stations_l = [generate_station(n) for n in np.random.randint(1,max_station+1, size=n_population)]

solution_pool = [Solution(stations_l[i]) for i in range(n_population)]

solution_df = pd.DataFrame({"solution_id":[x.id for x in solution_pool],
                            "solution_obj": solution_pool,
                            "from_generation":[-1]*n_population,
                            "from_method":["Randomize"]*n_population
                           })

### Iteration

In [17]:
iteration_best = []
iteration_pattern = []
iteration_df = []

In [18]:
solution_df = pd.read_pickle("solutions/solution-0100.pkl.bz2")

In [19]:
start = 101
num_iters = 1001
for i in tqdm(range(start, num_iters)):
    print(f"Iteration {i+1}/{num_iters}")
    
    data_pattern = np.random.choice(range(len(data)), p=p)
    # data_pattern = 0
    position = data[data_pattern]
    
    print(f"\tFitting with pattern {data_pattern}...")
    print(f"\tComputing fitness value...")
    solution_df = compute(solution_df, position)
    solution_df = solution_df.sort_values(by="fitness_value")
    
    new_solution_df = pd.DataFrame()
    if crossover_p:
        print(f"\tCrossover...")
        crossover_df = crossover(solution_df, crossover_p)
        crossover_df["from_method"] = ["Crossover"]*crossover_df.shape[0]
        new_solution_df = pd.concat([new_solution_df, crossover_df])
    if mutation_p:
        print(f"\tMutation...")
        mutation_df = mutation(solution_df, mutation_p)
        mutation_df["from_method"] = ["Mutation"]*mutation_df.shape[0]
        new_solution_df = pd.concat([new_solution_df, mutation_df])
        
    if new_solution_df.shape[0] != 0:
        print(f"\tComputing fitness value of new solutions...")
        new_solution_df["solution_id"] = new_solution_df["solution_obj"].parallel_apply(lambda x:x.id)
        new_solution_df = compute(new_solution_df, position)
        new_solution_df["from_generation"] = [i]*new_solution_df.shape[0]
    
    if elitism_ratio:
        print(f"\tPerforming elite selection and reinitialize...")
        n_elite = int(n_population * elitism_ratio)
        n_reinit = n_population - n_elite
        solution_df = solution_df.sort_values(by="fitness_value").head(n_elite)
        reinit_stations_l = [generate_station(n) for n in np.random.randint(1, max_station+1, size=n_reinit)]
        reinit_pool = [Solution(s) for s in reinit_stations_l]
        reinit_solution_df = pd.DataFrame({"solution_id":[x.id for x in reinit_pool],
                                          "solution_obj": reinit_pool
                                         })
        reinit_solution_df = compute(reinit_solution_df, position)
        reinit_solution_df["from_generation"] = [i]*reinit_solution_df.shape[0]
        reinit_solution_df["from_method"] = ["Randomize"]*reinit_solution_df.shape[0]
        solution_df = pd.concat([solution_df, reinit_solution_df], ignore_index=True)
    
    print(f"\tTrimming result...")
    if trim_policy=="random":
        print("\t\tRandom trim")
        solution_df = pd.concat([solution_df, new_solution_df], ignore_index=True).sample(n=n_population).reset_index(drop=True)
    else:
        print("\t\tRetain best trim")
        solution_df = pd.concat([solution_df, new_solution_df], ignore_index=True).sort_values(by="fitness_value").head(n_population).reset_index(drop=True)
    
    solution_df = solution_df.sort_values(by="fitness_value")
    best = deepcopy(solution_df[solution_df['fitness_value']==min(solution_df['fitness_value'])])
    iteration_best.append(best)
    iteration_pattern.append(data_pattern)
    iteration_df.append(deepcopy(solution_df[:10]))
    print(f"\tBest fitness of this iteration: {best['fitness_value'].iloc[0]}")
    if i%checkpoint_every == 0:
        fname = f"solutions/solution-{i:04d}.pkl.bz2"
        print(f"\tSaving checkpint at {fname}")
        solution_df.to_pickle(fname, compression="bz2")
    print(f"\tDone!")

  0%|          | 0/900 [00:00<?, ?it/s]

Iteration 102/1001
	Fitting with pattern 1...
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Performing elite selection and reinitialize...
	Trimming result...
		Retain best trim
	Best fitness of this iteration: 1.151731411892509
	Done!
Iteration 103/1001
	Fitting with pattern 1...
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Performing elite selection and reinitialize...
	Trimming result...
		Retain best trim
	Best fitness of this iteration: 1.151731411892509
	Done!
Iteration 104/1001
	Fitting with pattern 4...
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Performing elite selection and reinitialize...
	Trimming result...
		Retain best trim
	Best fitness of this iteration: 1.184789269913362
	Done!
Iteration 105/1001
	Fitting with pattern 3...
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value o

In [10]:
solution_df.groupby("from_generation").count()

,solution_id,solution_obj,from_method,signal_strength,construction_cost,num_orphans,fitness_value
from_generation,,,,,,,
51,1,1,1,1,1,1,1
60,1,1,1,1,1,1,1
61,1,1,1,1,1,1,1
63,3,3,3,3,3,3,3
65,1,1,1,1,1,1,1
66,1,1,1,1,1,1,1
67,1,1,1,1,1,1,1
68,2,2,2,2,2,2,2
69,1,1,1,1,1,1,1


In [11]:
iteration_df[-1]

,solution_id,solution_obj,from_generation,from_method,signal_strength,construction_cost,num_orphans,fitness_value
0,df1d9a81-a941-4600-82a5-f3566418c06f,Solution : df1d9a81-a941-4600-82a5-f3566418c06...,91,Crossover,-97.289179,10720000,0,1.152447
1,869a62ec-6a27-4f3b-b2ee-4492a54d380f,Solution : 869a62ec-6a27-4f3b-b2ee-4492a54d380...,100,Crossover,-97.487527,10720000,0,1.154250
2,abbaab70-7655-4588-95bf-4898dbf6f1a4,Solution : abbaab70-7655-4588-95bf-4898dbf6f1a...,60,Crossover,-98.421078,10440000,0,1.155737
3,783de3d3-21bf-4925-86ef-c1d58de553e8,Solution : 783de3d3-21bf-4925-86ef-c1d58de553e...,89,Crossover,-98.421078,10440000,0,1.155737
4,4b741601-3da5-4d6f-a506-974f247ce9f2,Solution : 4b741601-3da5-4d6f-a506-974f247ce9f...,98,Crossover,-96.073934,11330000,0,1.156649
5,98e34ae1-a24e-45b8-b13a-137392eb670f,Solution : 98e34ae1-a24e-45b8-b13a-137392eb670...,73,Mutation,-98.430555,10510000,0,1.157573
6,ff1edd87-67b8-4c88-a6ed-e8783721a8e5,Solution : ff1edd87-67b8-4c88-a6ed-e8783721a8e...,98,Crossover,-96.174098,11340000,0,1.157810
7,2b0370d6-c97e-475c-96ef-505a2c9b9704,Solution : 2b0370d6-c97e-475c-96ef-505a2c9b970...,84,Crossover,-97.925727,10720000,0,1.158234
8,cad9ebae-3ba1-4a77-a1b5-a66dfcf245b7,Solution : cad9ebae-3ba1-4a77-a1b5-a66dfcf245b...,91,Crossover,-98.085520,10720000,0,1.159687
9,e71e2727-099b-49fd-b073-fc399ed413bd,Solution : e71e2727-099b-49fd-b073-fc399ed413b...,89,Mutation,-98.799514,10510000,0,1.160927


In [12]:
pd.concat(iteration_best).to_pickle("iter_best-001.pkl")